In [22]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import imageio

In [23]:
# Render the frames figures
figs = []
n_frames = 8
for i in range(n_frames):
    print(f"Generating figures {i / n_frames * 100:5.2f}%")
    
    # Example data
    data = {'team': ['t1', 't2', 't3', 't4'],
            'lat': [49.0176 + 0.0001 * i, 49.0176 - 0.0001 * i, 49.0176, 49.0176],
            'lon': [12.0974, 12.0974, 12.0974 + 0.0001 * i, 12.0974 - 0.0001 * i]}
    df = pd.DataFrame(data)

    # Create plot
    fig = px.scatter_mapbox(df, lat="lat", lon="lon", color="team", mapbox_style="carto-positron", zoom=15)
    figs.append(fig)

Generating figures  0.00%
Generating figures 12.50%
Generating figures 25.00%
Generating figures 37.50%
Generating figures 50.00%
Generating figures 62.50%
Generating figures 75.00%
Generating figures 87.50%


In [24]:
# Convert figures to images
frames = []
for i, fig in enumerate(figs):
    print(f"Generating images {i / n_frames * 100:5.2f}%")

    img_bytes = pio.to_image(fig, format="png")
    img = imageio.v2.imread(img_bytes)
    frames.append(img)

Generating images  0.00%
Generating images 12.50%
Generating images 25.00%
Generating images 37.50%
Generating images 50.00%
Generating images 62.50%
Generating images 75.00%
Generating images 87.50%


In [25]:
# Save frames as a video
imageio.mimsave("plotly.mp4", frames, fps=1)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (700, 500) to (704, 512) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
